In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

# Ref: https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/efficientnet.py
# Install: pip install timm
import timm
from timm.models.efficientnet import default_cfgs 

from tensorboardX import SummaryWriter

import os
import shutil
import sys
import time
from glob import glob
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from torchvision import transforms as T

from sklearn.model_selection import train_test_split

device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
img_path = [i.replace('\\','/') for i in glob("./datasets/GenderDataset/*/*.jpg")]
genders = ['MAN', 'WOMAN', 'UNK']
df_data = pd.Series(img_path,name="img_path").to_frame()
df_data['gender'] = df_data['img_path'].apply(lambda x: genders.index(x.split('/')[-2]))
# df_data['gender'] = df_data['img_path'].apply(lambda x: x.split('/')[-2])

df = df_data[df_data.gender!=2].reset_index(drop=True)
df

/tmp/ipykernel_68/1966511220.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_data = pd.Series(img_path,name="img_path").to_frame()


,img_path,gender


In [3]:
(df['gender']==0).sum()

0

In [4]:
df_train, df_val = train_test_split(df, test_size=0.1, random_state=42)
df_train.shape,df_val.shape

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [5]:
# Dataset Loader
img_size = 192
augmentation = T.Compose([
    T.RandomResizedCrop(img_size, scale=(0.08, 1.)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
])
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self,df,img_size = img_size, data_aug = False):
        self.img_path = list(df['img_path'])
        self.label = list(df['gender'])
        self.img_size = img_size
        self.data_aug = data_aug
        self.default_augmentation = T.Compose([
                T.Resize((img_size,img_size)),
                T.ToTensor(),
            ])
    def __len__(self):
        return len(self.img_path)
    def __getitem__(self, idx):
        img = cv2.imread(self.img_path[idx])[:,:,::-1]
        img = Image.fromarray(img)
        if self.data_aug:
            img = self.data_aug(img)
        else:
            img = self.default_augmentation(img)
        return {'image': img, 'label': self.label[idx]}

train_dataset = ImageDataset(df_train, data_aug = augmentation)
val_dataset = ImageDataset(df_val)


# dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

NameError: name 'df_train' is not defined

In [6]:
class ClassifierModel(torch.nn.Module):
    def __init__(self,num_class=2):
        super(ClassifierModel, self).__init__() 
        self.effnv2_b0_model = timm.create_model(
                    'tf_efficientnetv2_b0',
                    pretrained=True)
        self.effnv2_b0_model.classifier = torch.nn.Linear(1280, num_class)


    def forward(self, x):
        x = self.effnv2_b0_model(x)
        return x

In [12]:
model = ClassifierModel()
model.effnv2_b0_model.conv_stem

Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)

In [14]:
from timm.models.layers.conv2d_same import Conv2dSame
Conv2dSame(6, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)

Conv2dSame(6, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)

In [7]:
# class ClassifierModel(torch.nn.Module):
#     def __init__(self,n_dim1 = 256,num_class=10,dropout_rate=0.3):
#         super(ClassifierModel, self).__init__() 
#         self.effnv2_b0_model = timm.create_model(
#                     'tf_efficientnetv2_b0',
#                     pretrained=True)
#         self.droupout1 = torch.nn.Dropout(p=dropout_rate)
#         self.droupout2 = torch.nn.Dropout(p=dropout_rate)

#         self.fc1 = torch.nn.Linear(1000, n_dim1)
#         self.fc2 = torch.nn.Linear(n_dim1, num_class)

#     def forward(self, x):
#         x = self.effnv2_b0_model(x)
#         x = self.droupout1(x)
#         x = self._mlp(x)
#         return x
    
#     def _mlp(self,x):
#         x = self.fc1(x)
#         x = torch.nn.functional.leaky_relu(x)
#         x = self.droupout2(x)
#         x = self.fc2(x)
#         return x


In [7]:
def stream(message) :
    try:
        sys.stdout.write("\r{%s}" % message)
    except:
        #Remove non-ASCII characters from message
        message = ''.join(i for i in message if ord(i)<128)
        sys.stdout.write("\r{%s}" % message)

def adjust_learning_rate(optimizer, epoch):
    LR_START = 1e-5
    LR_MAX = 1e-3
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 0
    LR_STEP_DECAY = 0.75
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = LR_MAX * LR_STEP_DECAY**((epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)//10)

    for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    return lr

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

def compute_accuracy(logit,label):
    pred_label = logit.argmax(axis=1)
    return (pred_label==label).sum()/logit.shape[0]
    
def compute_avg_w(l): # (data,num_batch)
    sum_w_loss = 0
    sum_w = 0
    for i in l:
        sum_w_loss+=i[0]*i[1]
        sum_w+=i[1]
    return sum_w_loss/sum_w

def train(model,train_loader,test_loader,cfg,pretrained_model_checkpoint = None):
    # model = DLA(num_classes=cfg.num_class).to(device)
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters())

    if pretrained_model_checkpoint:
        model.load_state_dict(torch.load(pretrained_model_checkpoint)['state_dict'])

    if not os.path.exists(cfg.output_directory):
        os.mkdir(cfg.output_directory)
    if cfg.save_to_tensorboard:
        logger = SummaryWriter(os.path.join(cfg.output_directory, 'logs'))

    

    best_epoch = 0
    best_val_loss = np.inf
    for epoch in range(cfg.epochs):
        print("Epoch: {}/{}".format(epoch+1,cfg.epochs))
        adjust_learning_rate(optimizer, epoch)
        # Training Model
        model.train()
        pred = []
        for i, batch in enumerate(train_loader):
            start_time = time.time()
            optimizer.zero_grad()
            img = batch['image'].to(device)
            label = batch['label'].to(device)
            logit = model(img)
            loss = criterion(logit,label)
            loss.backward()
            optimizer.step()
            num_step = i + len(train_loader) * epoch
            step_loss = loss.item()
            step_accuracy = compute_accuracy(logit,label).item()
            pred+=[(step_accuracy,label.shape[0])]
            msg = f"| Epoch: {epoch+1}/{cfg.epochs} ({i+1}/{len(train_loader)}) | Loss: {step_loss:#.4} | Accuracy: {step_accuracy:#.4} | {1./(time.time() - start_time):#.3} steps/s | Step: {num_step//1000}k |"
            stream(msg)
            if cfg.save_to_tensorboard:
                logger.add_scalar('Loss/train', step_loss, num_step)
                logger.add_scalar('Accuracy/train', step_accuracy, num_step)

        epoch_train_accuracy =compute_avg_w(pred)
        msg = f"\n\n| Train Accuracy: {epoch_train_accuracy:#.4} |"
        print(msg)

        # Evaluate Model
        model.eval()
        losses = []
        pred = []

        for i, batch in enumerate(test_loader):
            img = batch['image'].to(device)
            label = batch['label'].to(device)
            with torch.no_grad():
                logit = model(img)
                loss = criterion(logit,label)
            losses+=[(loss.item(),label.shape[0])]
            pred+=[(compute_accuracy(logit,label),label.shape[0])]

        epoch_val_loss = compute_avg_w(losses)
        epoch_val_accuracy =compute_avg_w(pred).item()

        msg = f"| Val Loss: {epoch_val_loss:#.4} | Val Accuracy: {epoch_val_accuracy:#.4} |\n"
        print(msg)
        if cfg.save_to_tensorboard:
            logger.add_scalar('Loss/val', epoch_val_loss, epoch)
            logger.add_scalar('Accuracy/val', epoch_val_accuracy, epoch)

        save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict(),
            }, is_best=False, filename=os.path.join(cfg.output_directory, 'checkpoint_{:04d}.pth.tar'.format(epoch)))
        
        # Save Best Checkpoint
        if best_val_loss>=epoch_val_loss:
            best_val_loss = epoch_val_loss
            best_epoch = epoch
            # Save Best Checkpoint
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'optimizer' : optimizer.state_dict(),
                }, is_best=False, filename=os.path.join(cfg.output_directory, 'best_checkpoint.pth.tar'.format(epoch)))

        # Early Stopping
        # Check Val loss until no improvement after which training
        # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
        if epoch-best_epoch>cfg.patience:
            break



In [10]:
class cfg:
    output_directory = "./weight/gender_cls_effnv2b0_checkpoint_from_MOCOv3_v2"
    epochs = 100
    num_class = 9
    # Early Stopping Params
    patience = 10 # Number of epochs with no improvement after which training will be stopped.
    save_to_tensorboard = True # tensorboard --logdir ./logs
model = ClassifierModel()
model.effnv2_b0_model.load_state_dict(torch.load('./weight/moco_checkpoint_gender_cls_v2/effnv2_b0_model_pretrained.pth.tar')['state_dict'] ,strict=False)
train(model,train_loader,test_loader,cfg)

Epoch: 1/100
{| Epoch: 1/100 (126/126) | Loss: 0.6525 | Accuracy: 0.6364 | 16.0 steps/s | Step: 0k |}

| Train Accuracy: 0.5565 |
| Val Loss: 0.6592 | Val Accuracy: 0.6323 |

Epoch: 2/100
{| Epoch: 2/100 (126/126) | Loss: 0.4877 | Accuracy: 0.7273 | 15.9 steps/s | Step: 0k |}

| Train Accuracy: 0.7172 |
| Val Loss: 0.4904 | Val Accuracy: 0.7590 |

Epoch: 3/100
{| Epoch: 3/100 (126/126) | Loss: 0.4280 | Accuracy: 0.7273 | 19.0 steps/s | Step: 0k |}

| Train Accuracy: 0.7679 |
| Val Loss: 0.4491 | Val Accuracy: 0.7937 |

Epoch: 4/100
{| Epoch: 4/100 (126/126) | Loss: 0.4439 | Accuracy: 0.8636 | 19.2 steps/s | Step: 0k |}

| Train Accuracy: 0.7794 |
| Val Loss: 0.5599 | Val Accuracy: 0.7152 |

Epoch: 5/100
{| Epoch: 5/100 (126/126) | Loss: 0.3833 | Accuracy: 0.8636 | 14.7 steps/s | Step: 0k |}

| Train Accuracy: 0.7865 |
| Val Loss: 0.5574 | Val Accuracy: 0.7186 |

Epoch: 6/100
{| Epoch: 6/100 (126/126) | Loss: 0.5530 | Accuracy: 0.6818 | 16.7 steps/s | Step: 0k |}

| Train Accuracy: 0.78

In [11]:
from line_notify import send_line_notify
send_line_notify('Finish Training Model')

'{"status":200,"message":"ok"}'

In [12]:
class cfg:
    output_directory = "./weight/gender_cls_effnv2b0_checkpoint_from_MOCOv3_v1"
    epochs = 100
    num_class = 9
    # Early Stopping Params
    patience = 10 # Number of epochs with no improvement after which training will be stopped.
    save_to_tensorboard = True # tensorboard --logdir ./logs
model = ClassifierModel()
model.effnv2_b0_model.load_state_dict(torch.load('./weight/moco_checkpoint_gender_cls_v1/effnv2_b0_model_pretrained.pth.tar')['state_dict'] ,strict=False)
train(model,train_loader,test_loader,cfg)

Epoch: 1/100
{| Epoch: 1/100 (126/126) | Loss: 0.6769 | Accuracy: 0.6364 | 18.4 steps/s | Step: 0k |}

| Train Accuracy: 0.5860 |
| Val Loss: 0.6432 | Val Accuracy: 0.6256 |

Epoch: 2/100
{| Epoch: 2/100 (126/126) | Loss: 0.4488 | Accuracy: 0.6818 | 17.2 steps/s | Step: 0k |}

| Train Accuracy: 0.7203 |
| Val Loss: 0.4767 | Val Accuracy: 0.7489 |

Epoch: 3/100
{| Epoch: 3/100 (126/126) | Loss: 0.4159 | Accuracy: 0.7273 | 18.8 steps/s | Step: 0k |}

| Train Accuracy: 0.7701 |
| Val Loss: 0.5054 | Val Accuracy: 0.7601 |

Epoch: 4/100
{| Epoch: 4/100 (126/126) | Loss: 0.5821 | Accuracy: 0.6818 | 17.7 steps/s | Step: 0k |}

| Train Accuracy: 0.7851 |
| Val Loss: 0.4417 | Val Accuracy: 0.7982 |

Epoch: 5/100
{| Epoch: 5/100 (126/126) | Loss: 0.5402 | Accuracy: 0.7727 | 17.9 steps/s | Step: 0k |}

| Train Accuracy: 0.7938 |
| Val Loss: 0.4798 | Val Accuracy: 0.7522 |

Epoch: 6/100
{| Epoch: 6/100 (126/126) | Loss: 0.3613 | Accuracy: 0.8636 | 16.3 steps/s | Step: 0k |}

| Train Accuracy: 0.79

In [13]:
from line_notify import send_line_notify
send_line_notify('Finish Training Model')

'{"status":200,"message":"ok"}'

In [30]:
model = ClassifierModel()
model.load_state_dict(torch.load('./weight/gender_cls_effnv2b0_checkpoint_from_MOCOv3_v1/best_checkpoint.pth.tar')['state_dict'] ,strict=False)

<All keys matched successfully>

In [31]:
pred_label = []
actual_label = []
model.cuda().eval()
for batch in test_loader:
    img = batch['image'].cuda()
    with torch.no_grad():
        pred = model(img).cpu().numpy().argmax(axis=1)
    pred_label += [pred]
    actual_label += [batch['label'].numpy()]

pred_label = np.concatenate(pred_label)        
actual_label = np.concatenate(actual_label)

In [32]:
(pred_label==actual_label).sum()/len(actual_label)

0.7982062780269058